In [225]:
from sheets import data, headers
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime

In [226]:
df = pd.read_csv("../data/matches.csv")
df["date"] = pd.to_datetime(df["date"])
df["city"] = df["city"].fillna("Dubai")
df["team1"] = df["team1"].replace("Rising Pune Supergiants", "Rising Pune Supergiant")
df["team2"] = df["team2"].replace("Rising Pune Supergiants", "Rising Pune Supergiant")
df["winner"] = df["winner"].replace("Rising Pune Supergiants", "Rising Pune Supergiant")
df["venue"] = df["venue"].replace("Rajiv Gandhi International Stadium, Uppal", "Rajiv Gandhi Intl. Cricket Stadium")
df["venue"] = df["venue"].replace("Punjab Cricket Association IS Bindra Stadium, Mohali", "Punjab Cricket Association Stadium")
df["venue"] = df["venue"].replace("Punjab Cricket Association Stadium, Mohali", "Punjab Cricket Association Stadium")
df["venue"] = df["venue"].replace("M Chinnaswamy Stadium", "M. Chinnaswamy Stadium")
df["venue"] = df["venue"].replace("Feroz Shah Kotla", "Feroz Shah Kotla Ground")
df["venue"] = df["venue"].replace("MA Chidambaram Stadium, Chepauk", "M. A. Chidambaram Stadium")
df["venue"] = df["venue"].replace("IS Bindra Stadium", "Punjab Cricket Association Stadium")
df["venue"] = df["venue"].replace("ACA-VDCA Stadium", "Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium")

df["city"] = df["city"].replace("Bangalore", "Bengaluru")

## Stadiums with wins when bowling first and wins with batting first

In [227]:
df["bat/field"] = "NO"
df.loc[(df["toss_winner"] == df["winner"]) & (df["toss_decision"] == "bat"), "bat/field"] = "bat"
df.loc[(df["toss_winner"] == df["winner"]) & (df["toss_decision"] == "field"), "bat/field"] = "field"
df.loc[(df["bat/field"] == "NO") & (df["toss_decision"] == "field"), "bat/field"] = "bat"
df.loc[(df["bat/field"] == "NO") & (df["toss_decision"] == "bat"), "bat/field"] = "field"

In [228]:
# df["venue"].value_counts()

# Rajiv Gandhi International Stadium, Uppal --> Rajiv Gandhi Intl. Cricket Stadium
# Punjab Cricket Association IS Bindra Stadium, Mohali --> Punjab Cricket Association Stadium, Mohali
# M Chinnaswamy Stadium --> M. Chinnaswamy Stadium
# Feroz Shah Kotla --> Feroz Shah Kotla Ground
# MA Chidambaram Stadium, Chepauk --> M. A. Chidambaram Stadium
df.columns 

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3', 'bat/field'],
      dtype='object')

In [229]:
df.loc[df["venue"]=="Buffalo Park", ["venue", "toss_winner", "toss_decision", "bat/field", "winner"]]

,venue,toss_winner,toss_decision,bat/field,winner
137,Buffalo Park,Mumbai Indians,bat,bat,Mumbai Indians
143,Buffalo Park,Chennai Super Kings,bat,bat,Chennai Super Kings
149,Buffalo Park,Mumbai Indians,bat,field,Delhi Daredevils


In [230]:
df_batfield = df.groupby(["venue", "bat/field"]).size().reset_index(name='counts')

In [231]:
df_bat = df_batfield.loc[df_batfield["bat/field"]=="bat"]
df_field = df_batfield.loc[df_batfield["bat/field"]=="field"]

In [232]:
df_bat.sort_values(by="counts", ascending=False, inplace=True)
df_field.sort_values(by="counts", ascending=False, inplace=True)

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [233]:
df_bat

,venue,bat/field,counts
65,Wankhede Stadium,bat,36
29,M. Chinnaswamy Stadium,bat,36
16,Feroz Shah Kotla Ground,bat,35
27,M. A. Chidambaram Stadium,bat,35
14,Eden Gardens,bat,30
43,Rajiv Gandhi Intl. Cricket Stadium,bat,28
41,Punjab Cricket Association Stadium,bat,24
49,Sawai Mansingh Stadium,bat,15
59,Subrata Roy Sahara Stadium,bat,11
25,Kingsmead,bat,8


## Try to take percentage of wins when batting first i.e. out of total matches played in a stadium how much percentage of it was won when batting / fielding first

In [234]:
# df_batfield["venue"].value_counts().reset_index(name="total matches")
# df_total_matches = df["venue"].value_counts().reset_index(name="total_matches")

In [235]:
df_total_matches = df_batfield.groupby(["venue"])["counts"].sum().reset_index(name="total_matches")

In [236]:
df_batfield2 = df_batfield.copy()
df_batfield2

,venue,bat/field,counts
0,Barabati Stadium,bat,4
1,Barabati Stadium,field,3
2,Brabourne Stadium,bat,6
3,Brabourne Stadium,field,5
4,Buffalo Park,bat,2
...,...,...,...
62,SuperSport Park,field,8
63,"Vidarbha Cricket Association Stadium, Jamtha",bat,2
64,"Vidarbha Cricket Association Stadium, Jamtha",field,1
65,Wankhede Stadium,bat,36


In [237]:
df_total_matches

,venue,total_matches
0,Barabati Stadium,7
1,Brabourne Stadium,11
2,Buffalo Park,3
3,De Beers Diamond Oval,3
4,Dr DY Patil Sports Academy,17
5,Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...,13
6,Dubai International Cricket Stadium,7
7,Eden Gardens,77
8,Feroz Shah Kotla Ground,74
9,Green Park,4


In [238]:
stad_matches = dict(zip(df_total_matches.venue, df_total_matches.total_matches))

In [239]:

for index, row in df_batfield2.iterrows():
    for stadium in stad_matches:
        if row.venue == stadium:
            df_batfield2.at[index, "total_m"] = int(stad_matches[stadium])
    

In [240]:
df_batfield2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 4 columns):
venue        67 non-null object
bat/field    67 non-null object
counts       67 non-null int64
total_m      67 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.2+ KB


In [241]:
df_batfield2["total_m"] = df_batfield2["total_m"].astype(int)

In [242]:
df_batfield2["perc_of_wins"] = round((df_batfield2["counts"]/df_batfield2["total_m"])*100,2)
# df_batfield2 = df_batfield2[:25]
df_batfield2

,venue,bat/field,counts,total_m,perc_of_wins
0,Barabati Stadium,bat,4,7,57.14
1,Barabati Stadium,field,3,7,42.86
2,Brabourne Stadium,bat,6,11,54.55
3,Brabourne Stadium,field,5,11,45.45
4,Buffalo Park,bat,2,3,66.67
...,...,...,...,...,...
62,SuperSport Park,field,8,12,66.67
63,"Vidarbha Cricket Association Stadium, Jamtha",bat,2,3,66.67
64,"Vidarbha Cricket Association Stadium, Jamtha",field,1,3,33.33
65,Wankhede Stadium,bat,36,73,49.32


In [243]:
fig  = px.bar(df_batfield2, y = "venue", x="perc_of_wins", orientation="h", color="bat/field", height=1000)
fig.show()

In [246]:
def bat_or_field(decision):
    sorted_bf = df_batfield2.loc[df_batfield2["bat/field"]==str(decision)][:11].sort_values(by="perc_of_wins", ascending=True)
    # return sorted_bf
    fig  = px.bar(sorted_bf, y = "venue", x="perc_of_wins", orientation="h", opacity=0.7, color_continuous_scale=["#ffffff", "#ff5f5f" ], labels={"venue":"Stadiums", "perc_of_wins":f"Win percentage with {decision} first"}, width=1000) #color="bat/field", height=1000
    fig.show()
    
decision="bat"
bat_or_field(decision)